In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import math
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm

import keras
from tensorflow import keras
from keras import Sequential
from keras import layers
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams['figure.dpi'] = 300
colors = ["#B6EE56", "#D85F9C", "#EEA756", "#56EEE8"]

# Loading the data

In [2]:
PATH_train = 'C:/Users/Krutika/Desktop/SEM-5-Projects/ML-Alzheimers/Alzheimer_s Dataset/train'
PATH_test = 'C:/Users/Krutika/Desktop/SEM-5-Projects/ML-Alzheimers/Alzheimer_s Dataset/test'

In [3]:
data = tf.keras.utils.image_dataset_from_directory(PATH_train,
                                                batch_size = 32,
                                                image_size=(128, 128),
                                                shuffle=True,
                                                seed=42,)

class_names = data.class_names

Found 4352 files belonging to 2 classes.


In [4]:
test_data = tf.keras.utils.image_dataset_from_directory(
    PATH_test,
    batch_size=32,
    image_size=(128, 128),
    shuffle=False  
)

Found 1088 files belonging to 2 classes.


In [5]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.9, val_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    
    return train_ds, val_ds


In [6]:
dataset_size = data.cardinality().numpy()

In [7]:
train_data, val_data = get_dataset_partitions_tf(data,dataset_size, train_split=0.9, val_split=0.1, shuffle=True, shuffle_size=10000)

# SVM

In [9]:
def dataset_to_numpy(dataset):
    images = []
    labels = []
    for image_batch, label_batch in dataset:
        for image in image_batch:
            images.append(image.numpy().reshape(-1))  # Flatten image
        labels.extend(label_batch.numpy())
    return np.array(images), np.array(labels)

In [10]:
train_images, train_labels = dataset_to_numpy(train_data)
test_images, test_labels = dataset_to_numpy(test_data)

In [11]:
train_images = train_images / 255.0  # Normalize pixel values
test_images = test_images / 255.0

In [12]:
classifier = svm.SVC(gamma=0.001)

In [13]:
train_images_flattened = train_images.reshape(len(train_images), -1)
test_images_flattened = test_images.reshape(len(test_images), -1)

In [14]:
classifier.fit(train_images_flattened, train_labels)

SVC(gamma=0.001)

In [19]:
# Predict labels for the test data
pred_labels = classifier.predict(test_images_flattened)

In [20]:
# print(classification_report(test_images_flattened,pred_target))
report = classification_report(test_labels, pred_labels)

In [22]:
print(report)

              precision    recall  f1-score   support

           0       0.82      0.76      0.79       640
           1       0.69      0.76      0.72       448

    accuracy                           0.76      1088
   macro avg       0.75      0.76      0.76      1088
weighted avg       0.77      0.76      0.76      1088



# CNN

In [28]:
def build_model():
    model = Sequential()

    model.add(Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), activation="relu", kernel_initializer='he_normal',
                     input_shape=(128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation="relu", kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), activation="relu", kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation="relu", kernel_initializer='he_normal'))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(2, activation="softmax"))

    model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

    model.summary()

    return model

model = build_model()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 63, 63, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 128)       36992     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 128)     

In [24]:
def checkpoint_callback():

    checkpoint_filepath = 'C:/Users/Krutika/Desktop/SEM-5-Projects/ML-Alzheimers/checkpoint'



    model_checkpoint_callback= ModelCheckpoint(filepath=checkpoint_filepath,
                           save_weights_only=False,
                           frequency='epoch',
                           monitor='val_accuracy',
                           save_best_only=True,
                           verbose=1)

    return model_checkpoint_callback

def early_stopping(patience):
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=1)
    return es_callback


EPOCHS = 18
checkpoint_callback = checkpoint_callback()
early_stopping = early_stopping(patience=5)
callbacks = [checkpoint_callback, early_stopping]

In [25]:
history = model.fit(train_data, epochs = EPOCHS, validation_data = val_data, class_weight = class_weights, callbacks = callbacks)

Epoch 1/18
122/122 [==============================] - ETA: 0s - loss: 17.1405 - accuracy: 0.6260
Epoch 1: val_accuracy improved from -inf to 0.88221, saving model to C:/Users/Krutika/Desktop/SEM-5-Projects/ML-Alzheimers\checkpoint


INFO:tensorflow:Assets written to: C:/Users/Krutika/Desktop/SEM-5-Projects/ML-Alzheimers\checkpoint\assets


INFO:tensorflow:Assets written to: C:/Users/Krutika/Desktop/SEM-5-Projects/ML-Alzheimers\checkpoint\assets


122/122 [==============================] - 32s 225ms/step - loss: 17.1405 - accuracy: 0.6260 - val_loss: 0.3125 - val_accuracy: 0.8822
Epoch 2/18
122/122 [==============================] - ETA: 0s - loss: 0.4602 - accuracy: 0.8102
Epoch 2: val_accuracy improved from 0.88221 to 0.93510, saving model to C:/Users/Krutika/Desktop/SEM-5-Projects/ML-Alzheimers\checkpoint


INFO:tensorflow:Assets written to: C:/Users/Krutika/Desktop/SEM-5-Projects/ML-Alzheimers\checkpoint\assets


...

INFO:tensorflow:Assets written to: C:/Users/Krutika/Desktop/SEM-5-Projects/ML-Alzheimers\checkpoint\assets


122/122 [==============================] - 27s 214ms/step - loss: 0.0773 - accuracy: 0.9693 - val_loss: 0.0221 - val_accuracy: 0.9928
Epoch 7/18
122/122 [==============================] - ETA: 0s - loss: 0.0259 - accuracy: 0.9915
Epoch 7: val_accuracy improved from 0.99279 to 0.99760, saving model to C:/Users/Krutika/Desktop/SEM-5-Projects/ML-Alzheimers\checkpoint


INFO:tensorflow:Assets written to: C:/Users/Krutika/Desktop/SEM-5-Projects/ML-Alzheimers\checkpoint\assets


INFO:tensorflow:Assets written to: C:/Users/Krutika/Desktop/SEM-5-Projects/ML-Alzheimers\checkpoint\assets


122/122 [==============================] - 27s 214ms/step - loss: 0.0259 - accuracy: 0.9915 - val_loss: 0.0134 - val_accuracy: 0.9976
Epoch 8/18
122/122 [==============================] - ETA: 0s - loss: 0.0501 - accuracy: 0.9821
Epoch 8: val_accuracy did not improve from 0.99760
122/122 [==============================] - 26s 202ms/step - loss: 0.0501 - accuracy: 0.9821 - val_loss: 0.0593 - val_accuracy: 0.9736
Epoch 9/18
122/122 [==============================] - ETA: 0s - loss: 0.2299 - accuracy: 0.9214
Epoch 9: val_accuracy did not improve from 0.99760
122/122 [==============================] - 26s 201ms/step - loss: 0.2299 - accuracy: 0.9214 - val_loss: 0.0604 - val_accuracy: 0.9712
Epoch 10/18
122/122 [==============================] - ETA: 0s - loss: 0.1241 - accuracy: 0.9572
Epoch 10: val_accuracy did not improve from 0.99760
122/122 [==============================] - 26s 201ms/step - loss: 0.1241 - accuracy: 0.9572 - val_loss: 0.0624 - val_accuracy: 0.9784
Epoch 11/18
122/122 [

In [26]:
model.evaluate(test_data)

34/34 [==============================] - 2s 44ms/step - loss: 1.4619 - accuracy: 0.7298


[1.4619392156600952, 0.7297794222831726]

In [27]:
predictions = []
labels = []

for X, y in test_data.as_numpy_iterator():
    y_pred = model.predict(X, verbose=0)
    y_prediction = np.argmax(y_pred, axis=1)
    predictions.extend(y_prediction)
    labels.extend(y)

predictions = np.array(predictions)
labels = np.array(labels)

print(classification_report(labels, predictions, target_names=class_names))

                  precision    recall  f1-score   support

     NonDemented       0.79      0.73      0.76       640
VeryMildDemented       0.66      0.72      0.69       448

        accuracy                           0.73      1088
       macro avg       0.72      0.73      0.72      1088
    weighted avg       0.74      0.73      0.73      1088

